In [3]:
import numpy as np
import cirq

In [4]:
# Kullback-Leibler divergence
from scipy.special import kl_div
kl_div(5, 4)

0.11571775657104855

In [9]:
# Create a few simple ansatze
from openfermioncirq import VariationalAnsatz 
from openfermioncirq.variational.letter_with_subscripts import LetterWithSubscripts

class BoundedAnsatz(VariationalAnsatz): 
    def param_bounds(self): 
        bounds = []
        for param in self.params():
            bounds.append((-2*np.pi, 2*np.pi))
        return bounds
    
class Idle(BoundedAnsatz):
    def __init__(self, num_qubits): 
        self.num_qubits = num_qubits 
        super().__init__(None)
    def params(self): 
        # Dummy parameter
        yield LetterWithSubscripts('dummy') 
    def _generate_qubits(self):
        return cirq.LineQubit.range(self.num_qubits)
    def operations(self, qubits): 
        for q in range(self.num_qubits):
            yield cirq.ops.I.on(self.qubits[q])

In [10]:
eye = Idle(1)
eye.circuit.to_text_diagram()

'0: ───I───'

In [27]:
def place_fid_into_bin(fid, n_bins): 
    # Return index of bin 0 to n_bins-1 of bin fid belongs in
    if fid > 1 or fid < 0: 
        raise ValueError("Fidelity cannot be greater than 1 or less than 0. Fidelity is {}".format(fid))
    if fid < 1.0/n_bins: return 0
    if fid == 1.0: return n_bins - 1
    return int(fid * n_bins)

In [30]:
def haar_fid(f, n):
    # Returns prob density of fidelity f
    return (n-1) * (1-f)**(n-2)

n_hist = 75 
n_params = 1
n_samples = 1000
hist = [0 for i in range(n_hist)]
params = np.linspace(0, 1, n_samples)

for i in range(n_samples):
    hist[place_fid_into_bin(params[i], n_hist)] += haar_fid(params[i], 2)/n_samples

In [ ]:
def uniform_params(ansatz): 
    # All params must have same bounds
    bounds = ansatz.param_bounds()
    #NOT DONE YET

In [60]:
import itertools
def uniformly_random_params(ansatz):
    # All params must have same bounds
    bounds = ansatz.param_bounds() 
    num_bounds = len(bounds)
    samples = 1000
    samples_per_bound = samples // num_bounds 
    params = []
    loop = range(samples_per_bound)
    for x in loop:
        p = [x / samples_per_bound * (bounds[0][1] - bounds[0][0]) + bounds[0][0]]
        params.append(p)
    return params

In [ ]:
# def gen_pdf(ansatz):
#     fid_pdf = {}
#     params = uniformly_random_params(ansatz)
#     for i in range(1000):
#         j = i / 1000 
#         fid_pdf[j] = fid_pdf.get(j, 0) + 

In [42]:
eye_prob = [0 for i in range(74)] + [1]
total_kl_div = 0
for i in range(len(hist)): 
    total_kl_div += kl_div(eye_prob[i], hist[i])
print(total_kl_div)

4.268697949366879
